In [162]:
import pandas as pd
import math
from random import * 
# import data sets that will be used
seeds = pd.read_csv("../csvs/kaggle/predictive/NCAATourneySeeds.csv")
reg_season = pd.read_csv("../csvs/kaggle/predictive/RegularSeasonDetailedResults.csv")
regions = pd.read_csv("../csvs/kaggle/predictive/Seasons.csv")
team_names = pd.read_csv("../csvs/kaggle/predictive/Teams.csv")
outcomes_14 = pd.read_csv("../csvs/1314/1314_outcomes.csv", encoding = 'latin-1')
outcomes_15 = pd.read_csv("../csvs/1415/1415_outcomes.csv", encoding = 'latin-1')
outcomes_16 = pd.read_csv("../csvs/1516/1516_outcomes.csv", encoding = 'latin-1')
outcomes_17 = pd.read_csv("../csvs/1617/1617_outcomes.csv", encoding = 'latin-1')
regular_season = pd.read_csv("../csvs/kaggle/regular_season_stats.csv", encoding = 'latin-1')

pd.set_option('display.max_rows',1755)

# adjust data sets to only 2014 and later
seeds = seeds[seeds.Season > 2013]
regions = regions[regions.Season > 2013]
reg_season = reg_season[reg_season.Season > 2013]

In [163]:
# Adds the column TeamName to the seeds dataframe
names = []
for team in seeds.TeamID:
    names.append(team_names['TeamName'][team_names['TeamID']==team].values[0])
seeds['TeamName'] = names

In [164]:
def getName(team, season):
    print("getName")
    return seeds['TeamName'][(seeds['Seed'] == team)&(seeds['Season']==season)].values[0]

def get_TeamID(team, season):
    return seeds['TeamID'][(seeds['Seed'] == team)&(seeds['Season']==season)].values[0]  

def get_Stat(team, season, indicator):
    return regular_season[(regular_season['TeamID']==team) & (regular_season['Season']==season)][indicator].values[0]

def prediction(team1, team2, i1, i2, season):
    t1 = get_TeamID(team1, season)
    t2 = get_TeamID(team2, season)
    
    t1_stat = get_Stat(t1, season, i1)*get_Stat(t1, season, i2)
    t2_stat = get_Stat(t2, season, i1)*get_Stat(t1, season, i2)
    
    if t1_stat>t2_stat:
        return team1, team2, 0
    else:
        return team2, team1, 1

def get_actual_results(season):
    actual_results = [[],[],[],[],[],[]]
    for round_num in range(1,7):
        #print('round_num = ' + str(round_num)) #DELETE
        num_teams = 2**(6-round_num)
        #print('num_teams = ' + str(num_teams)) #DELETE
        #print('printing outcomes_14.iloc[0:num_teams,round_num]') #DELETE
        #print(outcomes_14.iloc[0:num_teams,round_num]) #DELETE
        for i in range(num_teams):
            actual_results[round_num-1].append(outcomes_14.iloc[0:num_teams,round_num].values[i]) #TODO make it work for any year
    return actual_results

def get_tourney_results(season, indicators):
    tourney_order = []
    for x in open('order.txt','r'):
        tourney_order.append(x.strip())
    
    #resets array
    #results array is in format [roundof32, sweet16, elite8, final4, finals, ncaa_winner]
    roundof32, sweet16, elite8, final4, finals, ncaa_winner = [],[],[],[],[],[]
    tourney_results = [roundof32, sweet16, elite8, final4, finals, ncaa_winner]
    next_round = tourney_order[:]

    for round_num in range(0,6):        
        num_teams = 2**(6-round_num)
        for i in range(0, num_teams, 2):

            team1 = next_round[i]
            team2 = next_round[i+1]

            winner, loser, which = prediction(team1, team2,indicator1, indicator2, season)
            next_round.append(next_round[i+which])

            tourney_results[round_num].append(winner)

        del next_round[0:num_teams]

    return tourney_results

def get_points(tourney_results, actual_results):
    points, games_correct = 0,0
    for round_num in range(0,6):        
        num_teams = 2**(5-round_num)
        for i in range(num_teams):
            if tourney_results[round_num][i] == actual_results[round_num][i]:
                points += round_num
                games_correct += 1
    return points, games_correct

In [165]:
season = 2014
actual_results = get_actual_results(season)

for indicator1 in regular_season.iloc[:, 32:]: # iloc - integer locate - all rows, columns 3 to end
    for indicator2 in regular_season.iloc[:, 32:]:
        tourney_results = get_tourney_results(2014, [indicator1, indicator2])
        
        points, games_correct = get_points(tourney_results, actual_results)
       
        #print("points for this indictor ---->" + str(points))        
        if points > best_points: 
            best_points = points
            best_games = games_correct
            best_indicator = indicator1 + " " +indicator2
        
print("The best indicator is: %s" % best_indicator)
print("Max total points is equal to: %d" % 120)
print("Our Total points is equal to: %d" % best_points)
print("We predicted %d games correct out of %d games!" % (best_games, 63) )
print("This comes out to be about %d%% of all games" % round(best_games*100/63) )

The best indicator is: OPF OStl
Max total points is equal to: 120
Our Total points is equal to: 22
We predicted 16 games correct out of 63 games!
This comes out to be about 25% of all games
